In [ ]:
!pip install nltk

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
import bs4 as bs
import urllib.request
import re
import nltk
import heapq
#nltk.download("stopwords")

In [ ]:
source = urllib.request.urlopen("https://www.imperva.com/learn/application-security/web-scraping-attack/#:~:text=Web%20scraping%20is%20the%20process,replicate%20entire%20website%20content%20elsewhere.").read()
soup = bs.BeautifulSoup(source,'lxml')
text = ""

In [ ]:
#finding all the text that are in the paragraph tag in html.
#fetch all the text
for paragraph in soup.find_all('p'):
    text += paragraph.text
#preprocessing the text require re library
#removing all the number in the [] in the text
text=re.sub(r'\[[0-9]*\]',' ',text)
#remove extraspace
text=re.sub(r"\s+"," ",text)
clean_text=text.lower()
#remove the non-word character
clean_text=re.sub(r"\W"," ",clean_text)
#remove all digits
clean_text=re.sub(r"\d"," ",clean_text)
#remove the extra space
clean_text = re.sub(r"\s+"," ",clean_text)
#tokenise the whole paragraph to sentences wwe require nltk library

In [ ]:
sentences=nltk.sent_tokenize(text)
stop_words=nltk.corpus.stopwords.words("english")
#creating histogram where words-->keys and count-->values
word2count={}
for word in nltk.word_tokenize(clean_text):
    if word not in stop_words:
        if word not in word2count.keys():
            word2count[word]=1
        else:
            word2count[word]+=1
#converting counts to weights
max_count = max(word2count.values())
for key in word2count.keys():
    word2count[key] = word2count[key]/max_count

#create sentence score
sent2score = {}
for sentence in sentences:
    for word in nltk.word_tokenize(sentence.lower()):
        if word in word2count.keys():
            if len(sentence.split(" ")) < 25:
                if sentence not in sent2score.keys():
                    sent2score[sentence] = word2count[word]
                else:
                    sent2score[sentence] += word2count[word]
#print the number of sentence you want to display
#5 in parameter is the number of line you wanted to print
best_sent = heapq.nlargest(5, sent2score, key=sent2score.get)

In [ ]:
for sentence in best_sent:
    print(sentence)

Legitimate use cases include:Web scraping is also used for illegal purposes, including the undercutting of prices and the theft of copyrighted content.
The scraper can then replicate entire website content elsewhere.Web scraping is used in a variety of digital businesses that rely on data harvesting.
In price scraping, a perpetrator typically uses a botnet from which to launch scraper bots to inspect competing business databases.
Meanwhile, scraped sites often experience customer and revenue losses.Content scraping comprises large-scale content theft from a given site.
The two most common use cases are price scraping and content theft.


In [ ]:
import requests
from bs4 import BeautifulSoup
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords, wordnet
from nltk.probability import FreqDist
import random
import re
import nltk

# Download necessary resources
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

def get_text_from_url(url):
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise an exception for non-OK responses
        soup = BeautifulSoup(response.text, 'html.parser')

        # Content filtering: remove unwanted sections such as scripts, styles, and meta tags
        for unwanted_tag in soup(["script", "style", "meta"]):
            unwanted_tag.extract()

        # Get text from remaining paragraphs
        text = ' '.join([p.get_text() for p in soup.find_all('p')])
        return text.strip()  # Strip leading and trailing whitespace
    except requests.RequestException as e:
        print(f"Error fetching content from URL: {e}")
        return None
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return None

def get_summary(text, summary_length=3):
    sentences = sent_tokenize(text)
    word_frequencies = FreqDist(word_tokenize(text.lower()))
    most_frequent_words = [word for word, _ in word_frequencies.most_common(10)]
    summary = [sentence for sentence in sentences if any(word in sentence.lower() for word in most_frequent_words)]
    return ' '.join(summary[:summary_length])

def get_synonyms(word):
    synonyms = set()
    for syn in wordnet.synsets(word):
        synonyms.update(lemma.name() for lemma in syn.lemmas())
    return synonyms

def rephrase_sentence(sentence):
    words = re.findall(r'\b\w+\b', sentence)
    words = [word.lower() for word in words if word.isalnum() and word.lower() not in stopwords.words('english')]
    synonyms = [get_synonyms(word) or {word} for word in words]
    rephrased_sentence = [random.choice(list(synonym_set)) for synonym_set in synonyms]
    return ' '.join(rephrased_sentence)

def rephrase_summary(summary):
    sentences = sent_tokenize(summary)
    return ' '.join(rephrase_sentence(sentence) for sentence in sentences)

def main():
    url = input("Enter the URL: ")
    original_text = get_text_from_url(url)
    if original_text:
        summary_length = int(input("Enter the desired summary length (number of sentences): "))
        original_summary = get_summary(original_text, summary_length)
        print("\nOriginal Summary:")
        print(original_summary)
        rephrased_summary = rephrase_summary(original_summary)
        print("\nRephrased Summary:")
        print(rephrased_summary)
    else:
        print("Error fetching content from URL.")

if __name__ == "__main__":
    main()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


Enter the URL: https://en.wikipedia.org/wiki/Natural_language_processing
Enter the desired summary length (number of sentences): 2

Original Summary:
Natural language processing (NLP) is an interdisciplinary subfield of computer science and information retrieval. It is primarily concerned with giving computers the ability to support and manipulate human language.

Rephrased Summary:
raw words work NLP interdisciplinary subfield information_processing_system skill information recovery chiefly have-to_doe_with open computer ability plunk_for fake man words
